<a href="https://colab.research.google.com/github/hoky1227/bert_based-recommendation/blob/main/Transformer_berttokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install transformers
!pip install pytorch-transformers
# !pip install pytorchtools

     |████████████████████████████████| 2.0MB 8.3MB/s 
     |████████████████████████████████| 890kB 52.4MB/s 
     |████████████████████████████████| 3.2MB 53.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=c414a2acfbf46b69451935898e9cd95f5253a8fc58dfa0920835a3f8f2b8f80e
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 184kB 9.0MB/s 
     |████████████████████████████████| 1.2MB 8.5MB/s 
     |████████████████████████████████| 133kB 33.9MB/s 
     |████████████████████████████████| 7.3MB 34.5MB/s 
     |████████████████████████████████| 81kB 11.4MB/s 
ERROR: botocore 1.20.39 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.


In [3]:
# % pip install sentencepiece
# !pip install tokenizers

In [4]:
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [5]:
import tensorflow as tf
import torch
# import tensorflow_datasets as tfds

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
# import numpy as np
# import pandas as pd
# import torch
from torch.utils.data import Dataset
# from pytorch_transformers import BertTokenizer, BertForSequenceClassification, BertConfig
from torch.optim import Adam
import torch.nn.functional as F

import pandas as pd
import numpy as np
import random
import time
import datetime
import csv

In [6]:
path = '/content/drive/MyDrive/'

data_bert = pd.read_csv(path + 'train_bert.csv')
# data_space = pd.read_csv('/content/drive/MyDrive/train_space.csv')
Detail = pd.read_csv(path + 'Detail.csv')
data_bert.pop('Unnamed: 0')
# data_space.pop('Unnamed: 0')
Detail.pop('Unnamed: 0')
data_bert = data_bert.sample(frac=1).reset_index(drop=True)
data_bert.head()

,Customer,Detail,label
0,34606.0,DAISY FOLKART HEART DECORATION&&ROSE FOLKART H...,BASKET OF FLOWERS SEWING KIT
1,33874.0,BROWN CHECK CAT DOORSTOP&&CREAM CUPID HEARTS C...,WHITE HANGING HEART T-LIGHT HOLDER
2,35182.0,JUMBO BAG PINK WITH WHITE SPOTS&&JUMBO SHOPPER...,I'M ON HOLIDAY METAL SIGN
3,33891.0,PACK OF 60 PINK PAISLEY CAKE CASES&&PACK OF 72...,BAKING SET 9 PIECE RETROSPOT
4,38251.0,PACK OF 6 SKULL PAPER PLATES&&PACK OF 20 SKULL...,PINK SMALL GLASS CAKE STANDMAXWELL 2 TONE PINK...


In [7]:
data_bert.describe()

,Customer
count,11575.000000
mean,35354.741771
std,1694.022935
min,32346.000000
25%,33939.000000
50%,35298.000000
75%,36841.000000
max,38287.000000


In [8]:
print(len(data_bert))
# print(len(data_space))

11575


In [9]:
label_bert = data_bert['label'].values.tolist()
# label_space = data_space['label'].values.tolist()
print(len(label_bert))
# print(len(label_space))

11575


In [10]:
detail = Detail['0'].unique()
detail = detail.tolist()
len(detail)

3487

In [11]:
dic = {string : i for i,string in enumerate(detail)}
# print(dic)
len(dic)

3487

In [12]:
labels = []
for v in (label_bert):
    labels.append(dic.get(v))
print(labels)
print(len(labels))

[924, 661, 1442, 424, None, 424, 721, 647, 453, 400, 1299, 158, 283, 1169, 535, 1771, 48, 661, 396, 310, 432, 517, 2165, 180, 1226, 1089, 424, 1029, 727, 2891, 1375, 491, 2900, 1618, 1003, 109, 65, 202, 213, 40, 829, 1599, 199, 307, 243, 314, 1121, 1178, 310, 1668, 121, 1008, 3297, 749, 2253, 183, 1222, 665, 1019, 2196, None, 827, 870, 729, 348, 1457, 1300, 349, 121, 50, 2014, 1424, 2069, 1062, 402, 946, 67, 988, 1612, 342, 1183, 2349, 229, 299, 3033, 190, 1745, 161, 352, 231, 67, 514, 1966, 288, 1618, 727, 1803, 984, 1309, 951, 306, 1264, 657, 1299, 109, 200, 987, 1673, 90, 943, 1943, 117, 947, 158, 1406, 503, 124, 1686, 298, 1929, 1730, 663, 1152, 989, 749, 113, 190, 1822, 1188, 37, 965, 135, 55, 1086, 798, 1195, 2648, None, 765, 1735, 1223, 2403, 1584, 378, 1941, 937, 2223, 1903, 1688, 882, 1941, 186, 570, 649, 661, 29, 125, 361, 592, 158, 2008, 1909, 190, 177, 370, 473, 562, 840, 1229, 1677, 617, 383, 457, 1035, 2, 134, 550, 2306, 130, 1822, 34, 950, 1557, 550, 1143, None, 1103, 39

In [13]:
labelsdf = pd.DataFrame(labels)
data1 = pd.concat([data_bert, labelsdf], axis=1)
data1.dropna(inplace=True)
len(data1)

11259

In [14]:
data1.head()

,Customer,Detail,label,0
0,34606.0,DAISY FOLKART HEART DECORATION&&ROSE FOLKART H...,BASKET OF FLOWERS SEWING KIT,924.0
1,33874.0,BROWN CHECK CAT DOORSTOP&&CREAM CUPID HEARTS C...,WHITE HANGING HEART T-LIGHT HOLDER,661.0
2,35182.0,JUMBO BAG PINK WITH WHITE SPOTS&&JUMBO SHOPPER...,I'M ON HOLIDAY METAL SIGN,1442.0
3,33891.0,PACK OF 60 PINK PAISLEY CAKE CASES&&PACK OF 72...,BAKING SET 9 PIECE RETROSPOT,424.0
5,35323.0,PAPER BUNTING WHITE LACE&&VINTAGE UNION JACK S...,BAKING SET 9 PIECE RETROSPOT,424.0


In [15]:
labels = data1[0].values
labels = labels.tolist()
len(labels)

11259

In [16]:
print(labels[:5])

[924.0, 661.0, 1442.0, 424.0, 424.0]


In [17]:
cut = round(len(data1) * 0.8)
train, test = data1[:cut], data1[cut:]
train_label, test_label = data1[:cut][0], data1[cut:][0]
train_label, test_label = train_label.to_list(), test_label.to_list()

In [18]:
print(train_label)

[924.0, 661.0, 1442.0, 424.0, 424.0, 721.0, 647.0, 453.0, 400.0, 1299.0, 158.0, 283.0, 1169.0, 535.0, 1771.0, 48.0, 661.0, 396.0, 310.0, 432.0, 517.0, 2165.0, 180.0, 1226.0, 1089.0, 424.0, 1029.0, 727.0, 2891.0, 1375.0, 491.0, 2900.0, 1618.0, 1003.0, 109.0, 65.0, 202.0, 213.0, 40.0, 829.0, 1599.0, 199.0, 307.0, 243.0, 314.0, 1121.0, 1178.0, 310.0, 1668.0, 121.0, 1008.0, 3297.0, 749.0, 2253.0, 183.0, 1222.0, 665.0, 1019.0, 2196.0, 827.0, 870.0, 729.0, 348.0, 1457.0, 1300.0, 349.0, 121.0, 50.0, 2014.0, 1424.0, 2069.0, 1062.0, 402.0, 946.0, 67.0, 988.0, 1612.0, 342.0, 1183.0, 2349.0, 229.0, 299.0, 3033.0, 190.0, 1745.0, 161.0, 352.0, 231.0, 67.0, 514.0, 1966.0, 288.0, 1618.0, 727.0, 1803.0, 984.0, 1309.0, 951.0, 306.0, 1264.0, 657.0, 1299.0, 109.0, 200.0, 987.0, 1673.0, 90.0, 943.0, 1943.0, 117.0, 947.0, 158.0, 1406.0, 503.0, 124.0, 1686.0, 298.0, 1929.0, 1730.0, 663.0, 1152.0, 989.0, 749.0, 113.0, 190.0, 1822.0, 1188.0, 37.0, 965.0, 135.0, 55.0, 1086.0, 798.0, 1195.0, 2648.0, 765.0, 1735

In [19]:
len(data1['label'].unique())

1818

In [20]:
# import tensorflow_hub as hub
# FullTokenizer = bert.bert_tokenization.FullTokenizer
# bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1", trainable=False)

In [21]:
train['Detail'] = train['Detail'].str.replace('&&', ' [SEP] ')
sentences = train['Detail']
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[100]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


'[CLS] VICTORIAN GLASS HANGING T-LIGHT [SEP] HANGING HEART JAR T-LIGHT HOLDER [SEP] CREAM HEART CARD HOLDER [SEP]'

In [22]:
# # BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
# tokenized_texts_list = train['Detail'].to_list()
# tokenized_texts = []
# for v in tokenized_texts_list:
    # tokenized_texts.append([v])
print (sentences[0])
print (tokenized_texts[0])
len(tokenized_texts)


[CLS] DAISY FOLKART HEART DECORATION [SEP] ROSE FOLKART HEART DECORATIONS [SEP] JUNGLE POPSICLES ICE LOLLY HOLDERS [SEP] TOP SECRET PEN SET [SEP] STRAWBERRY CERAMIC TRINKET BOX [SEP] RECYCLING BAG RETROSPOT [SEP] BUTTERFLIES  HONEYCOMB GARLAND [SEP] YELLOW EASTER EGG HUNT START POST [SEP] BINGO SET [SEP] SILVER SKULL HOT WATER BOTTLE [SEP] VINTAGE SNAP CARDS [SEP] SET 12 RETRO WHITE CHALK STICKS [SEP] SWEETHEART CERAMIC TRINKET BOX [SEP] BIRD DECORATION GREEN  SPOT [SEP] HEART DECORATION WITH PEARLS [SEP] SKULLS DESIGN FLANNEL [SEP] PACK OF 72 SKULL CAKE CASES [SEP] PAPER BUNTING COLOURED LACE [SEP] CREAM FELT EASTER EGG BASKET [SEP] DECORATION , WOBBLY RABBIT , METAL [SEP] FELT EGG COSY BLUE RABBIT [SEP] HEART DECORATION RUSTIC HANGING [SEP] WHITE PEARL BEADED HEART, SMALL [SEP] WOBBLING METAL CHICKEN EASTER [SEP] FELT EGG COSY WHITE RABBIT [SEP] RIDGED GLASS T-LIGHT HOLDER [SEP] GARLAND WOODEN HAPPY EASTER [SEP] FELT EGG COSY CHICKEN [SEP] HANGING SPRING FLOWER EGG SMALL [SEP] BIRD 

7881

In [23]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 512

In [24]:
# tok = []
# for i in range(len(tokenized_texts)):
#     if len(tokenized_texts[i]) > 512:
#         tok.append(len(tokenized_texts[i]))
# print(max(tok))
# print(len(tok))
# print(len(tokenized_texts))

In [25]:
# for i in range(len(tokenized_texts)):
#     if len(tokenized_texts[i]) > 512:
#         while 1:
#             id = tokenized_texts[i].index('[SEP]')
#             tokenized_texts[i] = tokenized_texts[i][id:]
#             tokenized_texts[i][0] = '[CLS]'
#             if len(tokenized_texts[i]) <= 512:
#                 break
# tok = []
# for i in range(len(tokenized_texts)):
#     if len(tokenized_texts[i]) > 512:
#         tok.append(len(tokenized_texts[i]))
# # print(max(tok))
# print(len(tok))
# print(len(tokenized_texts))

In [26]:
for i in range(len(tokenized_texts)):
    if len(tokenized_texts[i]) > MAX_LEN:
        while 1:
            id = tokenized_texts[i].index('[SEP]')
            tokenized_texts[i] = tokenized_texts[i][id:]
            tokenized_texts[i][0] = '[CLS]'
            if len(tokenized_texts[i]) <= MAX_LEN:
                break
tok = []
for i in range(len(tokenized_texts)):
    if len(tokenized_texts[i]) > MAX_LEN:
        tok.append(len(tokenized_texts[i]))
# print(max(tok))
print(len(tok))
print(len(tokenized_texts))

0
7881


In [27]:
# 입력 토큰의 최대 시퀀스 길이
# MAX_LEN = 128

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

print(input_ids[0])
len(input_ids)

[  101 27137  2275   102  3165  7412  2980  2300  5835   102 13528 10245
  5329   102  2275  2260 22307  2317 16833 12668   102 12074 14692 13012
  8950  3388  3482   102  4743 11446  2665  3962   102  2540 11446  2007
 21944   102 21542  2640 13109 20147  2140   102  5308  1997  5824  7412
  9850  3572   102  3259 21122  3436 11401 12922   102  6949  2371 10957
  8288 10810   102 11446  1010 24185 24200 10442  1010  3384   102  2371
  8288  2522  6508  2630 10442   102  2540 11446 27471  5689   102  2317
  7247 26892  5732  2540  1010  2235   102 24185 15343  3384  7975 10957
   102  2371  8288  2522  6508  2317 10442   102  5526  2094  3221  1056
  1011  2422  9111   102 17017  4799  3407 10957   102  2371  8288  2522
  6508  7975   102  5689  3500  6546  8288  2235   102  4743 11446  2417
  3962   102  2235  2540  4870  8103   102  3259 21122  3436  2317 12922
   102  5689  3500  6546  8288  2312   102 15647 22619  5794  2094  1017
  7563   102  4918  1998 15137  4481  9543  2015   

7881

In [28]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,

In [29]:
# 훈련셋과 검증셋으로 분리
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    train_label, 
                                                                                    # random_state=2018, 
                                                                                    test_size=0.1)

# 어텐션 마스크를 훈련셋과 검증셋으로 분리
train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                    #    random_state=2018, 
                                                       test_size=0.1)

# 데이터를 파이토치의 텐서로 변환
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)				

print(train_inputs[0])
print(train_labels[0])
print(train_masks[0])
print(validation_inputs[0])
print(validation_labels[0])
print(validation_masks[0])

tensor([  101,  2665, 18353,  5603,  3286,  6546, 21545,  3482,   102,  2317,
         5689,  2540,  1056,  1011,  2422,  9111,   102,  4799,  4853, 14361,
         2317,   102,  6652,  3384,  2695, 11522,  3500,   102,  3536,  1016,
        13065,  5239,  2317,  3926,   102,  3536,  1055,  1013,  1017,  5239,
        14405,  2317,  3926,   102,  2417, 18353,  5603,  3286,  3123, 21545,
         3482,   102,  6949, 12074,  7163,  3108,   102, 21722, 15194,  6442,
         3104,   102, 21722,  6209,  8722,   102,  4610,  6209,  8722,   102,
         2034,  2465,  6209,  8722,   102,  2317, 12074,  2015,  5239,  1018,
        22497,   102,  2275,  2509,  2338,  3482,  2665, 18353,  5603,  3286,
         6546,   102,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0, 

In [30]:
# 배치 사이즈
batch_size = 2

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [31]:
test['Detail'] = test['Detail'].str.replace('&&', ' [SEP] ')
sentences = test['Detail']
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


['[CLS] PACK OF 20 SKULL PAPER NAPKINS [SEP] SMALL DOLLY MIX DESIGN ORANGE BOWL [SEP] UNION JACK FLAG PASSPORT COVER [SEP] LOO ROLL  METAL SIGN [SEP] SET/10 RED SPOTTY PARTY CANDLES [SEP] METAL SIGN,CUPCAKE SINGLE HOOK [SEP] 3D SHEET OF CAT STICKERS [SEP] SET/6 WOODLAND PAPER CUPS [SEP] CERAMIC CAKE STAND + HANGING CAKES [SEP] DOOR MAT TOPIARY [SEP] ANTIQUE SILVER TEA GLASS ETCHED [SEP] PINK FAIRY CAKE CUSHION COVER [SEP] CLASSIC FRENCH STYLE BASKET GREEN [SEP] MINATURE COLOURED GARDENING SET [SEP] BLUE FLOCK GLASS CANDLEHOLDER [SEP] BLACK FLOWER CANDLE PLATE [SEP] RED SPOTTY CHARLOTTE BAG [SEP] COLOUR GLASS T-LIGHT HOLDER HANGING [SEP] MULTI COLOUR SILVER T-LIGHT HOLDER [SEP] METAL SIGN CUPCAKE SINGLE HOOK [SEP] DOOR MAT FAIRY CAKE [SEP] METAL SIGN,CUPCAKE SINGLE HOOK [SEP] ANTIQUE LILY FAIRY LIGHTS [SEP] QUEEN OF THE SKIES PASSPORT COVER [SEP] EDWARDIAN PARASOL BLACK [SEP] EDWARDIAN PARASOL RED [SEP] SET/2 RED SPOTTY TEA TOWELS [SEP] JUMBO BAG CHARLIE AND LOLA TOYS [SEP] SILVER GLASS

In [32]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

[CLS] PACK OF 20 SKULL PAPER NAPKINS [SEP] SMALL DOLLY MIX DESIGN ORANGE BOWL [SEP] UNION JACK FLAG PASSPORT COVER [SEP] LOO ROLL  METAL SIGN [SEP] SET/10 RED SPOTTY PARTY CANDLES [SEP] METAL SIGN,CUPCAKE SINGLE HOOK [SEP] 3D SHEET OF CAT STICKERS [SEP] SET/6 WOODLAND PAPER CUPS [SEP] CERAMIC CAKE STAND + HANGING CAKES [SEP] DOOR MAT TOPIARY [SEP] ANTIQUE SILVER TEA GLASS ETCHED [SEP] PINK FAIRY CAKE CUSHION COVER [SEP] CLASSIC FRENCH STYLE BASKET GREEN [SEP] MINATURE COLOURED GARDENING SET [SEP] BLUE FLOCK GLASS CANDLEHOLDER [SEP] BLACK FLOWER CANDLE PLATE [SEP] RED SPOTTY CHARLOTTE BAG [SEP] COLOUR GLASS T-LIGHT HOLDER HANGING [SEP] MULTI COLOUR SILVER T-LIGHT HOLDER [SEP] METAL SIGN CUPCAKE SINGLE HOOK [SEP] DOOR MAT FAIRY CAKE [SEP] METAL SIGN,CUPCAKE SINGLE HOOK [SEP] ANTIQUE LILY FAIRY LIGHTS [SEP] QUEEN OF THE SKIES PASSPORT COVER [SEP] EDWARDIAN PARASOL BLACK [SEP] EDWARDIAN PARASOL RED [SEP] SET/2 RED SPOTTY TEA TOWELS [SEP] JUMBO BAG CHARLIE AND LOLA TOYS [SEP] SILVER GLASS T

In [33]:
# 입력 토큰의 최대 시퀀스 길이
# MAX_LEN = 128

In [34]:
for i in range(len(tokenized_texts)):
    if len(tokenized_texts[i]) > MAX_LEN:
        while 1:
            id = tokenized_texts[i].index('[SEP]')
            tokenized_texts[i] = tokenized_texts[i][id:]
            tokenized_texts[i][0] = '[CLS]'
            if len(tokenized_texts[i]) <= MAX_LEN:
                break
tok = []
for i in range(len(tokenized_texts)):
    if len(tokenized_texts[i]) > MAX_LEN:
        tok.append(len(tokenized_texts[i]))
# print(max(tok))
print(len(tok))
print(len(tokenized_texts))

0
3378


In [35]:
# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([  101,  5308,  1997,  2322,  7412,  3259, 20619,  2015,   102,
        2235, 19958,  4666,  2640,  4589,  4605,   102,  2586,  2990,
        5210, 12293,  3104,   102,  8840,  2080,  4897,  3384,  3696,
         102,  2275,  1013,  2184,  2417,  3962,  3723,  2283, 14006,
         102,  3384,  3696,  1010,  2452, 17955,  2309,  8103,   102,
        7605,  7123,  1997,  4937,  6293,  2545,   102,  2275,  1013,
        1020, 11051,  3259, 10268,   102, 14692,  9850,  3233,  1009,
        5689, 22619,   102,  2341, 13523,  2327, 17302,   102, 14361,
        3165,  5572,  3221, 20286,   102,  5061,  8867,  9850, 22936,
        3104,   102,  4438,  2413,  2806, 10810,  2665,   102, 19808,
       11244, 11401, 21529,  2275,   102,  2630, 19311,  3221, 13541,
       14528,   102,  2304,  6546, 13541,  5127,   102,  2417,  3962,
        3723,  5904,  4524,   102,  6120,  3221,  1056,  1011,  2422,
        9111,  5689,   102,  4800,  6120,  3165,  1056,  1011,  2422,
        9111,   102,

In [36]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,

In [37]:
# 데이터를 파이토치의 텐서로 변환
test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(test_label)
test_masks = torch.tensor(attention_masks)

print(test_inputs[0])
print(test_labels[0])
print(test_masks[0])

tensor([  101,  5308,  1997,  2322,  7412,  3259, 20619,  2015,   102,  2235,
        19958,  4666,  2640,  4589,  4605,   102,  2586,  2990,  5210, 12293,
         3104,   102,  8840,  2080,  4897,  3384,  3696,   102,  2275,  1013,
         2184,  2417,  3962,  3723,  2283, 14006,   102,  3384,  3696,  1010,
         2452, 17955,  2309,  8103,   102,  7605,  7123,  1997,  4937,  6293,
         2545,   102,  2275,  1013,  1020, 11051,  3259, 10268,   102, 14692,
         9850,  3233,  1009,  5689, 22619,   102,  2341, 13523,  2327, 17302,
          102, 14361,  3165,  5572,  3221, 20286,   102,  5061,  8867,  9850,
        22936,  3104,   102,  4438,  2413,  2806, 10810,  2665,   102, 19808,
        11244, 11401, 21529,  2275,   102,  2630, 19311,  3221, 13541, 14528,
          102,  2304,  6546, 13541,  5127,   102,  2417,  3962,  3723,  5904,
         4524,   102,  6120,  3221,  1056,  1011,  2422,  9111,  5689,   102,
         4800,  6120,  3165,  1056,  1011,  2422,  9111,   102, 

In [38]:
# 배치 사이즈
# batch_size = 32

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [39]:
# GPU 디바이스 이름 구함
device_name = tf.test.gpu_device_name()

# GPU 디바이스 이름 검사
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [40]:
# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [41]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [42]:
# 분류를 위한 BERT 모델 생성
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(detail))
model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [43]:
# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = 17

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 처음에 학습률을 조금씩 변화시키는 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [44]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()

    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [45]:
def intop10(logit, label, mode=0, top=10):
    '''mode = 0 (default) if you want to see boolean return'''
    l = []
    for _ in range(top):
        pred = np.argmax(logit)
        l.append(pred)
        logit[0][pred] = np.min(logit)
    if mode == 0:
        print(l)
        if label in l:
            return True
        else: return False
    elif mode == 1:
        if label in l:
            return 1
        else: return 0
    elif mode == 2:
        print(l)
        return l

In [46]:
def cal_acc(preds, label):
    acc = 0
    for v in preds:
        acc += intop10(v, label, mode=1)
    return acc / len(preds)

In [47]:
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [48]:
# 재현을 위해 랜덤시드 고정
# seed_val = 42
# random.seed(seed_val)
# np.random.seed(seed_val)
# torch.manual_seed(seed_val)
# torch.cuda.manual_seed_all(seed_val)



# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        b_labels = b_labels.type(torch.LongTensor).to(device)
        # print(b_labels.type())
        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    acc = 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1
        # acc = cal_acc(logits, label_ids)
    print("Accuracy: {0:.3f}".format(eval_accuracy/nb_eval_steps))
    # print("  Accuracy: {0:.3f}".format(acc))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 17 ========
Training...
  Batch   500  of  3,546.    Elapsed: 0:01:55.
  Batch 1,000  of  3,546.    Elapsed: 0:03:54.
  Batch 1,500  of  3,546.    Elapsed: 0:05:55.
  Batch 2,000  of  3,546.    Elapsed: 0:07:57.
  Batch 2,500  of  3,546.    Elapsed: 0:09:59.
  Batch 3,000  of  3,546.    Elapsed: 0:12:00.
  Batch 3,500  of  3,546.    Elapsed: 0:14:02.

  Average training loss: 7.56
  Training epcoh took: 0:14:13

Running Validation...
Accuracy: 0.013
  Validation took: 0:00:30

======== Epoch 2 / 17 ========
Training...
  Batch   500  of  3,546.    Elapsed: 0:02:01.
  Batch 1,000  of  3,546.    Elapsed: 0:04:03.
  Batch 1,500  of  3,546.    Elapsed: 0:06:05.
  Batch 2,000  of  3,546.    Elapsed: 0:08:06.
  Batch 2,500  of  3,546.    Elapsed: 0:10:08.
  Batch 3,000  of  3,546.    Elapsed: 0:12:09.
  Batch 3,500  of  3,546.    Elapsed: 0:14:11.

  Average training loss: 7.05
  Training epcoh took: 0:14:22

Running Validation...
Accuracy: 0.013
  Validation took: 0:00:3

In [49]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
acc = 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.3f}".format(eval_accuracy/nb_eval_steps))
    # acc = cal_acc(logits, label_ids)
# print("  Accuracy: {0:.3f}".format(acc)) # eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))

  Batch   100  of  1,689.    Elapsed: 0:00:08.
  Batch   200  of  1,689.    Elapsed: 0:00:15.
  Batch   300  of  1,689.    Elapsed: 0:00:23.
  Batch   400  of  1,689.    Elapsed: 0:00:31.
  Batch   500  of  1,689.    Elapsed: 0:00:39.
  Batch   600  of  1,689.    Elapsed: 0:00:47.
  Batch   700  of  1,689.    Elapsed: 0:00:54.
  Batch   800  of  1,689.    Elapsed: 0:01:02.
  Batch   900  of  1,689.    Elapsed: 0:01:10.
  Batch 1,000  of  1,689.    Elapsed: 0:01:18.
  Batch 1,100  of  1,689.    Elapsed: 0:01:25.
  Batch 1,200  of  1,689.    Elapsed: 0:01:33.
  Batch 1,300  of  1,689.    Elapsed: 0:01:41.
  Batch 1,400  of  1,689.    Elapsed: 0:01:49.
  Batch 1,500  of  1,689.    Elapsed: 0:01:56.
  Batch 1,600  of  1,689.    Elapsed: 0:02:04.

Accuracy: 0.012
Test took: 0:02:11


In [50]:
name = 'BERTmodel_bert_' + str(epochs) + '.pt'

torch.save(model, path + name)

In [51]:
# torch.save(model.state_dict(), './BERTmodel_stat.pt')

In [52]:
# 입력 데이터 변환
def convert_input_data(sentences):

    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 입력 토큰의 최대 시퀀스 길이
    MAX_LEN = 512
    # if len(tokenized_texts) > MAX_LEN:
    #     tokenized_texts = tokenized_texts[-Max_Len::]    
    for i in range(len(tokenized_texts)):
        if len(tokenized_texts[i]) > MAX_LEN:
            while 1:
                id = tokenized_texts[i].index('[SEP]')
                tokenized_texts[i] = tokenized_texts[i][id:]
                tokenized_texts[i][0] = '[CLS]'
                if len(tokenized_texts[i]) <= MAX_LEN:
                    break
    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

In [53]:
# 문장 테스트
def test_sentences(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits

In [54]:
# def kv(dic, val):
#     return [k for k, v in dic.items() if v == val]

prod_dic = {v:k for k,v in dic.items()}
# print(prod_dic)

def findprod(v):
    p = prod_dic.get(v)
    return p

In [55]:
test.reset_index(inplace=True)
test.pop('index')

0        8108
1        8109
2        8110
3        8111
4        8112
        ...  
3373    11570
3374    11571
3375    11572
3376    11573
3377    11574
Name: index, Length: 3378, dtype: int64

In [56]:
test.head()

,Customer,Detail,label,0
0,35311.0,PACK OF 20 SKULL PAPER NAPKINS [SEP] SMALL DOL...,SET OF RED SALAD SERVERS,1532.0
1,35527.0,SET OF 2 FANCY FONT TEA TOWELS [SEP] KINGS CHO...,BLACK RECORD COVER FRAME,357.0
2,32978.0,MEDIUM CHINESE STYLE SCISSOR [SEP] SMALL CHINE...,ABSTRACT CIRCLE JOURNAL,1163.0
3,33173.0,CERAMIC BIRDHOUSE FINCH BLUE LARGE [SEP] CERA...,HOT WATER BOTTLE TEA AND SYMPATHY,258.0
4,34415.0,COFFEE MUG CAT + BIRD DESIGN [SEP] LUNCH BAG R...,GREEN 3 PIECE MINI DOTS CUTLERY SET,458.0


In [57]:
model = torch.load(path + name)

In [58]:
acc = 0
for i in range(len(test)):
    logits = test_sentences([test['Detail'][i]])
    acc += intop10(logits, test[0][i], mode=1, top=5)
print("Accuracy: {0:.3f}".format(acc / len(test)))

Accuracy: 0.043


In [59]:
acc = 0
for i in range(len(test)):
    logits = test_sentences([test['Detail'][i]])
    acc += intop10(logits, test[0][i], mode=1)
print("Accuracy: {0:.3f}".format(acc / len(test)))

Accuracy: 0.070


In [60]:
acc = 0
for i in range(len(test)):
    logits = test_sentences([test['Detail'][i]])
    acc += intop10(logits, test[0][i], mode=1, top=15)
print("Accuracy: {0:.3f}".format(acc / len(test)))

Accuracy: 0.092


In [61]:
acc = 0
for i in range(len(test)):
    logits = test_sentences([test['Detail'][i]])
    acc += intop10(logits, test[0][i], mode=1, top=20)
print("Accuracy: {0:.3f}".format(acc / len(test)))

Accuracy: 0.112


In [62]:
acc = 0
for i in range(len(test)):
    logits = test_sentences([test['Detail'][i]])
    acc += intop10(logits, test[0][i], mode=1, top=25)
print("Accuracy: {0:.3f}".format(acc / len(test)))

Accuracy: 0.129


In [63]:
acc = 0
for i in range(len(test)):
    logits = test_sentences([test['Detail'][i]])
    acc += intop10(logits, test[0][i], mode=1, top=30)
print("Accuracy: {0:.3f}".format(acc / len(test)))

Accuracy: 0.145


In [64]:
# table = test.copy()
# for i in range(len(table)):
#     logits = test_sentences([table['Detail'][i]])
#     table['preds'][i] = intop10(logits, test[0][i], mode=2)

In [65]:
print("Hits: {}".format(acc))

Hits: 491


In [66]:
logits = test_sentences([test['Detail'][6]])

print(logits)
print((np.argmax(logits)))
print(len(logits[0]))
print(test[0][6])

[[ 1.1604877  -0.77230936  1.0132667  ... -3.2165518  -3.5134737
  -3.5878131 ]]
196
3487
645.0


In [67]:
logits = test_sentences([test['Detail'][100]])

print(logits)
print(np.argmax(logits))
# print(len(logits))
print(test['label'][100])
intop10(logits, test[0][100], mode=1)

[[ 1.1493559  1.3466905  2.7635603 ... -9.861267  -9.81743   -9.8647585]]
1640
CHRYSANTHEMUM NOTEBOOK


0

In [68]:
logits = test_sentences([data1['Detail'][100]])

# print(logits)
print(np.argmax(logits))
# print(len(logits))
print(data1[0][100])

195
306.0


In [69]:
a = []

for i in range(0, 10):
    logits = test_sentences([test['Detail'][i]])
    a.append(findprod(np.argmax(logits)))
asdf = test[:10].copy()
asdf['pred'] = a
asdf.pop(0)
asdf.pop('Customer')
asdf

,Detail,label,pred
0,PACK OF 20 SKULL PAPER NAPKINS [SEP] SMALL DOL...,SET OF RED SALAD SERVERS,SET OF 12 LILY BOTANICAL T-LIGHTS
1,SET OF 2 FANCY FONT TEA TOWELS [SEP] KINGS CHO...,BLACK RECORD COVER FRAME,CHERRY BLOSSOM DECORATIVE FLASK
2,MEDIUM CHINESE STYLE SCISSOR [SEP] SMALL CHINE...,ABSTRACT CIRCLE JOURNAL,GROW YOUR OWN FLOWERS SET OF 3
3,CERAMIC BIRDHOUSE FINCH BLUE LARGE [SEP] CERA...,HOT WATER BOTTLE TEA AND SYMPATHY,LIGHT PINK CHERRY LIGHTS
4,COFFEE MUG CAT + BIRD DESIGN [SEP] LUNCH BAG R...,GREEN 3 PIECE MINI DOTS CUTLERY SET,HOME BUILDING BLOCK WORD
5,TEA BAG PLATE RED SPOTTY [SEP] GARLAND WOODEN ...,GREY FLORAL FELTCRAFT SHOULDER BAG,I'M ON HOLIDAY METAL SIGN
6,"SPRING DEC , HANGING CHICK GREEN [SEP] GREEN ...",SET OF 36 MUSHROOM PAPER DOILIES,ROUND SNACK BOXES SET OF4 WOODLAND
7,PARTY METAL SIGN [SEP] TROPICAL LUGGAGE TAG [S...,SPRING FLOWER CHOPSTICKS SET/5,GARLAND WOODEN HAPPY EASTER
8,CREAM FELT EASTER EGG BASKET,PACK OF 6 BIRDY GIFT TAGS,PARTY BUNTING
9,SKULLS STICKERS [SEP] SET/4 SKULL BADGES [SEP...,BOX OF 24 COCKTAIL PARASOLS,WHITE WOOD GARDEN PLANT LADDER


In [70]:
for i in range(0, len(test), 500):
    logits = test_sentences([test['Detail'][i]])
    # print(np.argmax(logits))
    print(intop10(logits, test[0][i], mode=0))

[1088, 1686, 2131, 547, 64, 1606, 495, 407, 3022, 2143]
False
[317, 130, 1735, 989, 199, 144, 321, 1299, 1526, 1598]
False
[792, 294, 438, 619, 195, 109, 620, 452, 189, 55]
True
[139, 1062, 783, 988, 1686, 106, 46, 1571, 58, 577]
False
[196, 310, 391, 424, 322, 31, 298, 399, 299, 970]
False
[400, 661, 947, 121, 649, 827, 130, 914, 190, 615]
False
[207, 457, 190, 1081, 989, 180, 130, 453, 947, 400]
False


In [71]:
# print(findprod(661))
# print(findprod(400))

In [72]:
a = test['Detail'][199]
logits = test_sentences([a])
print('sent:',a)
b = intop10(logits, test[0][199], mode=2)
print('label:', findprod(test[0][199]), test[0][199])
p = []
for v in b:
    p.append(findprod(v))
print(p)

sent: 6 RIBBONS RUSTIC CHARM [SEP] UNION JACK FLAG LUGGAGE TAG [SEP] RED HEART LUGGAGE TAG [SEP] CLOTHES PEGS RETROSPOT PACK 24 [SEP] 20 DOLLY PEGS RETROSPOT [SEP] RED SPOTTY LUGGAGE TAG [SEP] ZINC WILLIE WINKIE  CANDLE STICK [SEP] 12 PENCILS TALL TUBE RED SPOTTY [SEP] TEA BAG PLATE RED SPOTTY [SEP] STRAWBERRY SCENTED SET/9 T-LIGHTS [SEP] CINAMMON SET OF 9 T-LIGHTS [SEP] ORANGE SCENTED SET/9 T-LIGHTS [SEP] S/3 FRENCH VANILLA SQU CANDLE [SEP] 12 PENCILS SMALL TUBE RED SPOTTY [SEP] PAPER BUNTING RETRO SPOTS [SEP] WOODEN SCHOOL COLOURING SET [SEP] HANGING HEART ZINC T-LIGHT HOLDER [SEP] NATURAL SLATE RECTANGLE CHALKBOARD [SEP] 72 SWEETHEART FAIRY CAKE CASES [SEP] LAVENDER SCENTED FABRIC HEART [SEP] PACK OF 72 RETRO SPOT CAKE CASES [SEP] HANGING JAM JAR T-LIGHT HOLDER [SEP] SET/20 RED SPOTTY PAPER NAPKINS [SEP] GROW YOUR OWN BASIL IN ENAMEL MUG [SEP] ZINC METAL HEART DECORATION [SEP] RED ROSE AND LACE C/COVER [SEP] SET 3 WICKER LOG BASKETS [SEP] FAIRY CAKE WICKER PICNIC BASKET [SEP] SET/5 

In [73]:
data1['label'].value_counts()

WHITE HANGING HEART T-LIGHT HOLDER    157
DOOR MAT UNION FLAG                    64
REGENCY CAKESTAND 3 TIER               53
HOME BUILDING BLOCK WORD               51
PARTY BUNTING                          50
                                     ... 
PAINTED LIGHTBULB STAR+ MOON            1
MINI PAINTED GARDEN DECORATION          1
ORIGAMI ROSE INCENSE IN TUBE            1
ORANGE SCENTED SET/9 T-LIGHTS           1
SPOTS MUG                               1
Name: label, Length: 1818, dtype: int64

In [74]:
data1[0].value_counts()

661.0     157
400.0      64
130.0      53
121.0      51
649.0      50
         ... 
2352.0      1
1284.0      1
2393.0      1
3346.0      1
1953.0      1
Name: 0, Length: 1818, dtype: int64

모델 참고 링크

https://zzaebok.github.io/deep_learning/nlp/Bert-for-classification/

https://huggingface.co/transformers/model_doc/bert.html

https://colab.research.google.com/drive/1tIf0Ugdqg4qT7gcxia3tL7und64Rv1dP#scrollTo=VJ76KiP_dLn-&uniqifier=1

In [75]:
# class NsmcDataset(Dataset):
#     ''' Naver Sentiment Movie Corpus Dataset '''
#     def __init__(self, df):
#         self.df = df

#     def __len__(self):
#         return len(self.df)

#     def __getitem__(self, idx):
#         text = self.df.iloc[idx, 0]
#         label = self.df.iloc[idx, 1]
#         return text, label

In [76]:
# la = pd.DataFrame(labels)
# display(la.head())
# train_data = pd.concat([train, la], axis=1)
# train_data.head()

In [77]:
# nsmc_train_dataset = NsmcDataset(train_data)
# train_loader = DataLoader(nsmc_train_dataset, batch_size=2, shuffle=True, num_workers=2)

In [78]:
# device = torch.device("cuda")
# tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
# model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=len(detail))
# model.to(device)

BERT

In [79]:
# !git clone https://github.com/google-research/bert.git

In [80]:
# !python /content/bert/create_pretraining_data.py \
#   --input_file=train \
#   --output_file=/tmp/tf_examples.tfrecord \
#   --vocab_file=$BERT_BASE_DIR/vocab.txt \
#   --do_lower_case=False \
#   --max_seq_length=128 \
#   --max_predictions_per_seq=20 \
#   --masked_lm_prob=0.15 \
#   --random_seed=12345 \
#   --dupe_factor=5

In [81]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [82]:
# optimizer = Adam(model.parameters(), lr=1e-6)

# itr = 1
# p_itr = 500
# epochs = 1
# total_loss = 0
# total_len = 0
# total_correct = 0


# model.train()
# for epoch in range(epochs):
    
#     for text, label in train_loader:
#         optimizer.zero_grad()
        
#         # encoding and zero padding
#         # encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
#         encoded_list = tokenized_texts
#         # padded_list =  [e + [0] * (512-len(e)) for e in encoded_list]
#         # 토큰을 숫자 인덱스로 변환
#         input_ids = [tokenizer.convert_tokens_to_ids(x) for x in encoded_list]

#         # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
#         padded_list = pad_sequences(input_ids, maxlen=256, dtype="long", truncating="post", padding="post")
#         # print(padded_list[0])
#         # print(len(padded_list[0]))
#         # print(len(padded_list))
#         # print(type(padded_list))
#         sample = torch.tensor(padded_list).to(torch.int32)
#         sample, label = sample.to(device), label.to(device)
#         labels = torch.tensor(label).to(torch.int32)
#         outputs = model(sample, labels=labels)
#         loss, logits = outputs

#         pred = torch.argmax(F.softmax(logits), dim=1)
#         correct = pred.eq(labels)
#         total_correct += correct.sum().item()
#         total_len += len(labels)
#         total_loss += loss.item()
#         loss.backward()
#         optimizer.step()
        
#         if itr % p_itr == 0:
#             print('[Epoch {}/{}] Iteration {} -> Train Loss: {:.4f}, Accuracy: {:.3f}'.format(epoch+1, epochs, itr, total_loss/p_itr, total_correct/total_len))
#             total_loss = 0
#             total_len = 0
#             total_correct = 0

#         itr+=1

In [83]:
# prompt = "In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced."
# next_sentence = "The sky is blue due to the shorter wavelength of blue light."
# encoding = tokenizer(prompt, next_sentence, return_tensors='pt')

# outputs = model(**encoding, labels=torch.LongTensor([1]))
# logits = outputs.logits
# assert logits[0, 0] < logits[0, 1] # next sentence was random